## Assignment 2 as a notebook

In [141]:
#Import of packages
import os
import sys
sys.path.append("..")

# Import teaching utils
import numpy as np
import utils.classifier_utils as clf_util
import cv2
import pandas as pd
from utils.imutils import jimshow
from utils.imutils import jimshow_channel

# Import sklearn metrics
from sklearn import metrics
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

#Dataset
from tensorflow.keras.datasets import cifar10

# Visualisation
import matplotlib.pyplot as plt

In [39]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [80]:
def X_preproces(data, dim, name):
    output_array = np.empty((1, dim*dim))

    for i, img in enumerate(data):
        img_g = cv2.cvtColor(data[i], cv2.COLOR_BGR2GRAY)
        img_gr = img_g.reshape(-1, dim*dim)
        img_grn = img_gr/255.0
        output_array = np.append(output_array, img_grn, axis=0) 

    outpath = os.path.join("..", "out", "X_" + name + "_preprocessed.npy")
    np.save(outpath, output_array)
    return output_array



def X_load(filename):
    X_file = np.load(filename)
    X_file = np.delete(X_file, 0, 0) #For some reason my np.array is lengthened by one. I have identified the first element to be the issue

    return X_file



def create_models(X_train_p, y_train, model):
    if model == 0:
        clf = LogisticRegression(tol=0.1, 
                        solver='saga',
                        multi_class='multinomial').fit(X_train_p, y_train)
    
    elif model == 1:
        clf = MLPClassifier(activation = "logistic",
                           hidden_layer_sizes = (20,),
                           max_iter=1000,
                           random_state = 42).fit(X_train_p, y_train)
        
    return clf
    

def save_cls_reports_and_plots(model, X_test, y_test, lbl_names, path_spec):
    y_pred = model.predict(X_test)

    cr = pd.DataFrame(metrics.classification_report(y_test, y_pred, output_dict=True)).transpose()
    outpath = os.path.join("..", "out", "classification_report" + path_spec + ".csv")
    cr.to_csv(outpath, index= True)


    plt.plot(model.loss_curve_)
    plt.title("Loss curve during training", fontsize=14)
    plt.xlabel('Iterations')
    plt.ylabel('Loss score')
    plt.show()
    plt.plot(model.loss_curve_)
    outpath = os.path.join("..", "out", "loss_curve" + path_spec + ".png")
    plt.savefig(outpath)

    return "Finished"

In [123]:
#Example to check why the dataset was lengthened to 50001..
a = cv2.cvtColor(X_train[0], cv2.COLOR_BGR2GRAY)
a = a.reshape(1, 1024)
a = a/255.0

print(a, X_train_p[0])

[[0.24313725 0.17647059 0.18431373 ... 0.68235294 0.43921569 0.35294118]] [0.24313725 0.17647059 0.18431373 ... 0.68235294 0.43921569 0.35294118]
